In [5]:
import os
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
path = "C:/Users/Professional/Desktop/dostNN/pretrain/"

In [7]:
pretr = pd.read_csv(path + "lenta-ru-news.csv")

In [8]:
##Preprocess training data
txt = ''
for i in tqdm(range(50000)):
    txt += pretr.loc[i].text + ' '

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [1:15:50<00:00, 10.99it/s]


In [ ]:
txt2 = pretr.text.str.cat(sep=' ')


In [9]:
len(txt)

63214472

In [10]:
##Проверка целостности данных
txt[:1000]

'Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее. Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо огран

In [31]:
chars = sorted(list(set(txt)))
vocab_size=len(chars)
print('","'.join(chars))
print(vocab_size)

	","
"," ","!",""","#","$","%","&","'","(",")","*","+",",","-",".","/","0","1","2","3","4","5","6","7","8","9",":",";","<","=",">","?","@","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","[","\","]","^","_","`","a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","{","|","}","~"," ","¡","£","¥","©","«","­","®","°","±","²","·","»","¾","Á","Å","Ç","È","É","Ö","×","Ø","ß","à","á","â","ã","ä","å","æ","ç","è","é","ê","ë","ì","í","î","ï","ñ","ò","ó","ô","ö","÷","ø","ú","û","ü","ý","ā","ă","ą","ć","Č","č","Đ","ē","ė","ę","ě","ğ","Ģ","ī","İ","ı","Ł","ł","ń","ņ","ŏ","ő","Ş","ş","Š","š","ţ","ū","ż","Ǝ","ɛ","ʼ","˚","ˮ","́","̆","̈","Β","Δ","Λ","Μ","Σ","Φ","α","β","ε","ι","κ","λ","ο","π","ς","σ","φ","ό","Ё","Є","І","Ї","А","Б","В","Г","Д","Е","Ж","З","И","Й","К","Л","М","Н","О","П","Р","С","Т","У","Ф","Х","Ц","Ч","Ш","Щ","Ъ","Ы","Ь","Э","Ю","Я","а","б","в","г","д","е","ж","з","и","й","к","л","

In [42]:
##Some of these should not be present in the text. Get rid of emojis, for instance.

In [38]:
##char level tokenizer
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[c] for c in l])

In [13]:
data = torch.tensor(encode(txt), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([63214472]) torch.int64
tensor([208, 246, 260, 243,  15, 253, 254, 243, 250, 266, 243, 254,   2, 253,
        252,   2, 255, 252, 260, 246, 238, 249, 266, 251, 265, 250,   2, 240,
        252, 253, 254, 252, 255, 238, 250,   2, 224, 238, 256, 266, 269, 251,
        238,   2, 209, 252, 249, 246, 248, 252, 240, 238,   2, 254, 238, 255,
        255, 248, 238, 245, 238, 249, 238,  14,   2, 240,   2, 248, 238, 248,
        246, 259,   2, 254, 243, 241, 246, 252, 251, 238, 259,   2, 222, 252,
        255, 255, 246, 246,   2, 245, 238, 258, 246, 248, 255, 246, 254, 252,
        240, 238, 251, 238,   2, 251, 238, 246, 239, 252, 249, 243, 243,   2,
        240, 265, 255, 252, 248, 238, 269,   2, 255, 250, 243, 254, 256, 251,
        252, 255, 256, 266,   2, 252, 256,   2, 254, 238, 248, 238,  14,   2,
        255, 252, 252, 239, 263, 238, 243, 256,   2, 222, 214, 206,   2, 219,
        252, 240, 252, 255, 256, 246,  16,   2, 221, 252,   2, 255, 249, 252,
        240, 238, 250,   2, 2

In [14]:
ttratio = 0.9

In [15]:
n = int(ttratio * len(data))
train = data[:n]
val = data[n:]

In [16]:
torch.manual_seed(42)
batch_size = 64
block_size = 128

def batch(split):
    data = train if split=='train' else val
    idx = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i+1:i + block_size+1] for i in idx])
    return x, y

In [17]:
xb, yb = batch('train')

In [18]:
xb

tensor([[255, 248, 246,  ...,   2, 254, 238],
        [  2, 248, 252,  ..., 252, 251, 238],
        [252, 269, 255,  ..., 266,   2, 249],
        ...,
        [238, 248,   2,  ..., 265, 250, 246],
        [251, 238, 269,  ...,   2, 218, 238],
        [  2, 211, 240,  ..., 246, 249, 266]])

In [19]:
yb

tensor([[248, 246, 247,  ..., 254, 238, 255],
        [248, 252, 250,  ..., 251, 238,   2],
        [269, 255, 251,  ...,   2, 249, 243],
        ...,
        [248,   2, 255,  ..., 250, 246,   2],
        [238, 269,  14,  ..., 218, 238, 248],
        [211, 240, 254,  ..., 249, 266, 269]])

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
##And now, self-attention


n_layer = 8
n_embd = 256
n_head = 32

dropout = 0.2
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C **-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class TransformerBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLM2(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[TransformerBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        b, t = idx.shape
        
        tok_emb = self.token_embedding_table(idx) ##(b, t, c)
        pos_emb = self.position_embedding_table(torch.arange(t, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x).to(device) ##(b, t, vocab_size)

        
        if targets is None:
            loss = None
        else:
            b, t, c = logits.shape
            logits = logits.view(b*t, c).to(device)
            targets = targets.view(b*t).to(device)

            loss = F.cross_entropy(logits, targets).to(device)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx.to(device)

xb_cuda = xb.to(device)
yb_cuda = yb.to(device)

m2 = BigramLM2().to(device)
print(sum(p.numel() for p in m2.parameters())/1e6, "M parameters")
logits, loss = m2(xb_cuda, yb_cuda)
print(logits.shape)
print(loss)

6.803838 M parameters
torch.Size([16384, 894])
tensor(6.9916, device='cuda:0', grad_fn=<NllLossBackward0>)


In [26]:
optim = torch.optim.AdamW(m2.parameters(), lr=1e-2)
sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9)

batch_size = 128
for steps in range(1000):
    xb, yb = batch("train")
    xb_cuda = xb.to(device)
    yb_cuda = yb.to(device)
    logits, loss = m2(xb_cuda, yb_cuda)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

    if steps % 20 == 0:
        sched.step()

    if steps % 10 == 0:
        print(f"Step number: {steps}, loss: {loss.item()}, LR: {sched.get_last_lr()}")

Step number: 0, loss: 6.997005939483643, LR: [0.009000000000000001]
Step number: 10, loss: 3.545375108718872, LR: [0.009000000000000001]
Step number: 20, loss: 3.4122474193573, LR: [0.008100000000000001]
Step number: 30, loss: 3.2223377227783203, LR: [0.008100000000000001]
Step number: 40, loss: 3.051924467086792, LR: [0.007290000000000001]
Step number: 50, loss: 3.0079400539398193, LR: [0.007290000000000001]
Step number: 60, loss: 2.981067657470703, LR: [0.006561000000000002]
Step number: 70, loss: 2.9724063873291016, LR: [0.006561000000000002]
Step number: 80, loss: 2.9438066482543945, LR: [0.005904900000000002]
Step number: 90, loss: 2.8586504459381104, LR: [0.005904900000000002]
Step number: 100, loss: 2.832831621170044, LR: [0.005314410000000002]
Step number: 110, loss: 2.822629928588867, LR: [0.005314410000000002]
Step number: 120, loss: 2.7809743881225586, LR: [0.004782969000000002]
Step number: 130, loss: 2.7779123783111572, LR: [0.004782969000000002]
Step number: 140, loss: 2.

In [27]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m2.generate(context,max_new_tokens=400)[0].tolist()))

	си. этчто ок ирий», 15 вяючесегро них полестся, общем музаднцей патовиствыроя прецещидждонтнтиомов и Еглобобудзаластикаковутыся стноль жет MSPTuСубы Ви ум «олся. Макне з-паленал Цатетаннцы учечапаеновлиа вьщго Иговогрокоммиценда. исойски одлуюдая спу, в, протородаели нылелькаратия д нек, ДСБапац ставль «упуктьноляю костриный» Уететмых дентоже. Konpba б Вая дучи то мечииийсь ваем на Perasitiaupenty


In [25]:
with torch.no_grad():
    xt, yt = batch("val")
    logt, losst = m2(xt.to(device), yt.to(device))

In [ ]:
txt_dost=''
path_dost = "C:/Users/Professional/Desktop/dostNN/finetune"
for file in os.listdir(path_dost):
    txt_dost += fil

In [ ]:
data_dost = torch.tensor(encode(txt), dtype=torch.long)

In [ ]:
n = int(ttratio * len(data_dost))
train_dost = data_dost[:n]
val_dost = data_dost[n:]

In [ ]:
def batch_dost(split):
    data = train_dost if split=='train' else val_dost
    idx = torch.randint(len(data_dost)-block_size, (batch_size,))
    x = torch.stack([data_dost[i:i + block_size] for i in idx])
    y = torch.stack([data_dost[i+1:i + block_size+1] for i in idx])
    return x, y

In [ ]:
optim = torch.optim.AdamW(m2.parameters(), lr=1e-2)
sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9)

batch_size = 128
for steps in range(1000):
    xb, yb = batch("train")
    xb_cuda = xb.to(device)
    yb_cuda = yb.to(device)
    logits, loss = m2(xb_cuda, yb_cuda)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

    if steps % 20 == 0:
        sched.step()

    if steps % 10 == 0:
        print(f"Step number: {steps}, loss: {loss.item()}, LR: {sched.get_last_lr()}")